In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import requests
from bs4 import BeautifulSoup



In [2]:
def divslooper(lis):
    returner = ''
    for i,val in enumerate(lis):
        returner += val
        if(i < (len(lis)-1)):
            returner += ' '
        
        
    return(returner)
    

In [3]:

def scrape(csvlink):
   
    with open(csvlink, 'r') as f:
        reader = csv.reader(f)
        your_list = list(reader)
        your_list.pop(0)

    name_list = list()
    for x in your_list:
        if len(x[0].split('&')) == 2:
            
            xf = x[0].split('&')
            name_list.append(xf[0]+'%26'+xf[1])
        else:   
            name_list.append(x[0])
    
    url_list = list()
    for x in name_list:
        url = 'https://www.set.or.th/set/factsheet.do?symbol=' + x + '&ssoPageId=3&language=en&country=US'
        url_list.append(url)
        
    rname_list  = list()
    for i in your_list:
        rname_list.append(i[0])
        
    
    scrape_list = [url_list,rname_list]
    return(scrape_list)
    


In [128]:
def tocsv(urllist,stock):
    
    sector_list = list()
    for i,x in enumerate(urllist):
        page = requests.get(x)
        soup = BeautifulSoup(page.text, 'html.parser')

#         general_list = soup.find(class_='factsheet')

#         general_list_item = general_list.find_all('td')

#         fact_list = list()
#         for y in general_list_item:
#              fact_list.append((str((y.contents[0].string))))
        
#         real_fact_list = list()
#         for j in range(18):
#             if j > 8:
#                 real_fact_list.append(fact_list[j])

#         real_fact_list.pop(1)
#         real_fact_list.pop(3)
#         real_fact_list.pop(4)
#         real_fact_list.pop(4)
        
 #         ========================================================================
#         flaw_ind = False 
        shareEq_list = list()
        eq_mean = None
        netProfit = None
        totalLb = None
        sales = None
        cogs = None
        rev = None
        first_signal = 0
        for z in soup.findAll('table',{'class':'factsheet'}):
            for tr in z.findAll('tr'):
                stuff=[td for td in tr.stripped_strings]
                if first_signal == 0:
                    first_signal+=1
                elif first_signal == 1:
                    price_list = stuff
                    first_signal = 2
        real_fact_list = price_list
        if len(real_fact_list) == 9:
            real_fact_list.pop(1)
            real_fact_list.pop(3)
            real_fact_list.pop(4)
            real_fact_list.pop(4)
        elif len(real_fact_list) == 6:
            
            real_fact_list = [real_fact_list[0],real_fact_list[2],real_fact_list[3],real_fact_list[5],'-']
        
        real_stat = list()
        cash_list = list()
        eps_list = list()
        growth_list = list()
        
        tf_weird = False  
        for z in soup.findAll('table',{'class':'factsheet'}):
            for tr in z.findAll('tr'):
                stuff=[td for td in tr.stripped_strings]
                
                if len(stuff) > 1 and stuff[0] =='BVPS (B./Share)':
                    real_stat.append(stuff[1])
                    try:
                        real_stat.append(stuff[2])
                    except IndexError:
                        real_stat.append(None)
                if len(stuff) > 1 and stuff[0] =='Payout Ratio':
                    real_stat.append(stuff[1])
                    try:
                        real_stat.append(stuff[2])
                    except IndexError:
                        real_stat.append(None)
                if len(stuff) > 1 and stuff[0] == 'Beta':
                    real_stat.append(stuff[1])
                    try:
                        real_stat.append(stuff[2])
                    except IndexError:
                        real_stat.append(None)
                        
                     
                
                if len(stuff) > 1 and stuff[0] == 'Statement of Financial Position (MB.)':
                    m_count = 0
                    
                    if len(stuff[1].split('/')) > 1:
                        tf_weird = True
                        for index1,tdf in enumerate(stuff):
                            if index1 > 0:
                                if list(tdf)[1] == 'M':
                                    m_count+= 1
                        print(stock[i], 'has an M in financial statement')
                        print(m_count)
                        
                
                        
                if tf_weird == False:   
                    if len(stuff) > 1 and stuff[0] == 'Cash':
                        if len(stuff) > 2:
                            if len(stuff) > 3:
                                if len(stuff) > 4:
                                    cash_list = [stuff[4],stuff[3],stuff[2],stuff[1]]
                                else:
                                    cash_list=[None,stuff[3],stuff[2],stuff[1]]
                            else: 
                                cash_list = [None,None,stuff[2],stuff[1]]
                        else: 
                            cash_list = [None,None,None,stuff[1]]


                    elif len(stuff) > 2 and stuff[0] == 'Cash & Deposits':
                        if len(stuff) > 3:
                            if len(stuff) > 4:
                                cash_list = [stuff[4],stuff[3],stuff[2],stuff[1]]
                            else: 
                                cash_list = [None,stuff[3],stuff[2],stuff[1]]
                        else: 
                            cash_list = [None,None,stuff[2],stuff[1]]


                    if len(stuff) > 1 and stuff[0] == 'EPS (B.)':
                        if len(stuff) > 2:
                            if len(stuff) > 3:
                                if len(stuff) > 4:
                                    eps_list = [stuff[4],stuff[3],stuff[2],stuff[1]]
                                else:
                                    eps_list=[None,stuff[3],stuff[2],stuff[1]]
                            else: 
                                eps_list = [None,None,stuff[2],stuff[1]]
                        else: 
                            eps_list = [None,None,None,stuff[1]]



                    if len(stuff) > 1 and stuff[0] == 'Net Profit Growth':
                        if len(stuff) > 2:
                            if len(stuff) > 3:
                                growth_list = [stuff[3],stuff[2],stuff[1]]
                            else:
                                growth_list = [None,stuff[2],stuff[1]]
                        else:
                            growth_list = [None,None,stuff[1]]



                    if len(stuff) > 0 and stuff[0] == "Shareholders' Equity":
                        shareEq_list = [float(stuff[1].replace(',','')),float(stuff[2].replace(',',''))]
                        eq_mean = (shareEq_list[0]+shareEq_list[1])/2
                    if len(stuff) > 0 and stuff[0] == 'Net Profit':
                        netProfit = float(stuff[1].replace(',',''))
                    if len(stuff) > 0 and stuff[0] == 'Total Liabilities':
                        totalLb = float(stuff[1].replace(',',''))
                    if len(stuff) > 0 and stuff[0] == 'Sales':
                        try:
                            sales = float(stuff[1].replace(',',''))
                        except ValueError:
                            print(stock[i] + 'has an error with sales')
                            sales = None
                    if len(stuff) > 0 and stuff[0] == 'COGs':
                        try: 
                            cogs = float(stuff[1].replace(',',''))
                        except ValueError:
                            print(stock[i] + 'has an error with cogs')
                            cogs = None
                    if len(stuff) > 0 and stuff[0] == 'Total Revenues':
                        try: 
                            rev = float(stuff[1].replace(',',''))
                        except ValueError:
                            print(stock[i] + 'has an error with Total Revenues')
                            rev = None


                    if netProfit != None and eq_mean != None:
                        roe = (netProfit/eq_mean) * 100
                    else:
                        roe = None
            #             print(stock[i] + 'no roe')

                    if totalLb != None and len(shareEq_list) != 0:

                        try:
                            de = totalLb / shareEq_list[0]
                        except IndexError:
                            print(stock[i],' has an index problem with d/e')
                    else:
                        de = None

                    if sales != None and cogs != None:
                        grosspm = (sales - cogs)/sales * 100
                    else:
                        grosspm = None

            #         try:
            #             grosspm = (sales - cogs)/sales * 100
            #         except TypeError:

            #             grosspm = None

            #         except UnboundLocalError:
            #             print(stock[i] + ' has unboundlocal error')
            #             grosspm = None

                    if netProfit != None and rev != None:
                        netpm = (netProfit/rev) * 100
                    else:
                        netpm = None

                    ratio_list = [roe,de,grosspm,netpm]

                    if len(growth_list) == 0:
                        growth_list = [None,None,None]

                    if len(cash_list) == 0:
                        cash_list = [None,None,None,None]
                    if len(eps_list) == 0:
                        eps_list = [None,None,None,None]
                
                else:
                    if len(stuff) > 1 and stuff[0] == 'Cash':
                        length = len(stuff) - m_count 
                        if m_count == 2:
                            if length > 2:
                                if length > 3:
                                    if length > 4:
                                        cash_list = [stuff[6],stuff[5],stuff[4],stuff[3]]
                                    else:
                                        cash_list=[None,stuff[5],stuff[4],stuff[3]]
                                else: 
                                    cash_list = [None,None,stuff[4],stuff[3]]
                            else: 
                                cash_list = [None,None,None,stuff[3]]
                                
                    elif len(stuff) > 2 and stuff[0] == 'Cash & Deposits':
                        length = len(stuff) - m_count 
                        if length > 3:
                            if length > 4:
                                cash_list = [stuff[6],stuff[5],stuff[4],stuff[3]]
                            else:
                                cash_list=[None,stuff[5],stuff[4],stuff[3]]
                        else: 
                            cash_list = [None,None,stuff[4],stuff[3]]
                            
                    if len(stuff) > 1 and stuff[0] == 'EPS (B.)':
                        length = len(stuff) - m_count 
                        if m_count == 2:
                            if length > 2:
                                if length > 3:
                                    if length > 4:
                                        eps_list = [stuff[6],stuff[5],stuff[4],stuff[3]]
                                    else:
                                        eps_list=[None,stuff[5],stuff[4],stuff[3]]
                                else: 
                                    eps_list = [None,None,stuff[4],stuff[3]]
                            else: 
                                eps_list = [None,None,None,stuff[3]]
                                
                    if len(stuff) > 1 and stuff[0] == 'Net Profit Growth':
                        length = len(stuff) - m_count 
                        if length > 2:
                            if length > 3: 
                                growth_list = [stuff[5],stuff[4],stuff[3]]
                            else:
                                growth_list = [None,stuff[4],stuff[3]]
                        else:
                            growth_list = [None,None,stuff[3]]
                    
                    if m_count == 2:
                            
                        if len(stuff) > 0 and stuff[0] == "Shareholders' Equity":
                            shareEq_list = [float(stuff[3].replace(',','')),float(stuff[4].replace(',',''))]
                            eq_mean = (shareEq_list[0]+shareEq_list[1])/2
                        if len(stuff) > 0 and stuff[0] == 'Net Profit':
                            netProfit = float(stuff[3].replace(',',''))
                        if len(stuff) > 0 and stuff[0] == 'Total Liabilities':
                            totalLb = float(stuff[3].replace(',',''))
                        if len(stuff) > 0 and stuff[0] == 'Sales':
                            try:
                                sales = float(stuff[3].replace(',',''))
                            except ValueError:
                                print(stock[i] + 'has an error with sales')
                                sales = None
                        if len(stuff) > 0 and stuff[0] == 'COGs':
                            try: 
                                cogs = float(stuff[3].replace(',',''))
                            except ValueError:
                                print(stock[i] + 'has an error with cogs')
                                cogs = None
                        if len(stuff) > 0 and stuff[0] == 'Total Revenues':
                            try: 
                                rev = float(stuff[3].replace(',',''))
                            except ValueError:
                                print(stock[i] + 'has an error with Total Revenues')
                                rev = None


                    if netProfit != None and eq_mean != None:
                        roe = (netProfit/eq_mean) * 100
                    else:
                        roe = None
            #             print(stock[i] + 'no roe')

                    if totalLb != None and len(shareEq_list) != 0:

                        try:
                            de = totalLb / shareEq_list[0]
                        except IndexError:
                            print(stock[i],' has an index problem with d/e')
                    else:
                        de = None

                    if sales != None and cogs != None:
                        grosspm = (sales - cogs)/sales * 100
                    else:
                        grosspm = None

            #         try:
            #             grosspm = (sales - cogs)/sales * 100
            #         except TypeError:

            #             grosspm = None

            #         except UnboundLocalError:
            #             print(stock[i] + ' has unboundlocal error')
            #             grosspm = None

                    if netProfit != None and rev != None:
                        netpm = (netProfit/rev) * 100
                    else:
                        netpm = None

                    ratio_list = [roe,de,grosspm,netpm]

                    if len(growth_list) == 0:
                        growth_list = [None,None,None]

                    if len(cash_list) == 0:
                        cash_list = [None,None,None,None]
                    if len(eps_list) == 0:
                        eps_list = [None,None,None,None]
                        
                    
                    
# ==================================================
        
        divc15 = list()
        divs15 = list()
        divc16 = list()
        divs16 = list()
        divc17 = list()
        divs17 = list()
        divc18 = list()
        divs18 = list()
        
        agg = 0
        divYear = ''

        for z in soup.findAll('table',{'class':'factsheet'}):
            for tr in z.findAll('tr'):
                stuff=[td for td in tr.stripped_strings]
                date_list = list()
                if len(stuff) > 2 and agg == 1:
                    if stuff[0] != 'Auditor':
                        
                        date_list= stuff[3].split(' ')
#                         if len(date_list) > 0 and date_list[2] == '2014':
#                             if stuff[2] == 'Baht':
#                                 divc14.append(float(stuff[1]))
#                             else:
#                                 divs14.append(stuff[1])
                        if len(date_list) > 0 and date_list[0] == '-':
                            if stuff[2] == 'Baht':
                                if divYear=='2015':
                                    divc15.append(float(stuff[1]))
                                elif divYear=='2016':
                                    divc16.append(float(stuff[1]))
                                elif divYear=='2017':
                                    divc17.append(float(stuff[1]))
                                elif divYear=='2018':
                                    divc18.append(float(stuff[1]))
                            else:
                                if divYear=='2015':
                                    divs15.append(stuff[1])
                                elif divYear=='2016':
                                    divs16.append(stuff[1])
                                elif divYear=='2017':
                                    divs17.append(stuff[1])
                                elif divYear=='2018':
                                    divs18.append(stuff[1])
                                    
                                
               
                        elif len(date_list) > 2 and date_list[2] == '2015':
                            divYear = '2015'
                            if stuff[2] == 'Baht':
                                divc15.append(float(stuff[1]))
                            else:
                                divs15.append(stuff[1])
                        elif len(date_list) > 2 and date_list[2] == '2016':
                            divYear = '2016'
                            if stuff[2] == 'Baht':
                                divc16.append(float(stuff[1]))
                            else:
                                divs16.append(stuff[1])
                        elif len(date_list) > 2 and date_list[2] == '2017':
                            divYear = '2017'
                            if stuff[2] == 'Baht':
                                divc17.append(float(stuff[1]))
                            else:
                                divs17.append(stuff[1])
                        elif len(date_list) > 2 and date_list[2] == '2018':
                            divYear = '2018'
                            if stuff[2] == 'Baht':
                                divc18.append(float(stuff[1]))
                            else:
                                divs18.append(stuff[1])
                    else: 
                        agg=0
                            
                if len(stuff) > 0 and stuff[0] == 'Operation Period':
                    agg+=1
        
#         divc14 = np.array(divc14)
#         divc14 = str(np.sum(divc14))
        divc15 = np.array(divc15)
        divc15 = str(np.sum(divc15))
        divc16 = np.array(divc16)
        divc16 = str(np.sum(divc16))
        divc17 = np.array(divc17)
        divc17 = str(np.sum(divc17))
        divc18 = np.array(divc18)
        divc18 = str(np.sum(divc18))

#         divs14 = divslooper(divs14)
        divs15 = divslooper(divs15)
        divs16 = divslooper(divs16)
        divs17 = divslooper(divs17)
        divs18 = divslooper(divs18)
        
        dividend = [divc15,divs15,divc16,divs16,divc17,divs17,divc18,divs18]
            
        if len(dividend) == 0:
            dividend = [None,None,None,None,None,None,None,None]
#         print(cash_list)
#         print(eps_list)
                    
        ultimate_list = real_fact_list + real_stat + dividend + cash_list + eps_list + ratio_list + growth_list
#         print(ultimate_list)
        temp_name = [stock[i]]
        ultimate_list = temp_name + ultimate_list
       
        
        for b,val in enumerate(ultimate_list):
            if val == '-' or val ==  ' ' or val == '' or val == '0.0':
                ultimate_list[b] = None
            if type(val) == str: 
                ultimate_list[b] = val.replace(',', '')
        sector_list.append(ultimate_list)
        taishi = pd.DataFrame(sector_list,columns = ['Stock','Price (B.)','P/E (X)','P/BV (X)','Market Cap (MB.)',
                                'EV/EBITDA','BVPS (B.) (2018)','BVPS (B.) (2017)', 'Beta (2018)','Beta (2017)',
                                'Payout Ratio (2018)','Payout Ratio (2017)','Dividend 15 (Cash)','Dividend 15 (Share)',
                                'Dividend 16 (Cash)','Dividend 16 (Share)','Dividend 17 (Cash)',
                                 'Dividend 17 (Share)', 'Dividend 18 (Cash)','Dividend 18 (Share)',
                                'Cash 14','Cash 15','Cash 16','Cash 17','EPS 14 (baht)','EPS 15 (baht)',
                                'EPS 16 (baht)','EPS 17 (baht)','ROE (%)','D/E (X)','Gross Profit Margin (%)',
                                'Net Profit Margin (%)','Net Profit Growth 15','Net Profit Growth 16','Net Profit Growth 17'])
   
    return(taishi)

In [214]:
scrplist = scrape('C:/Users/GE62/Desktop/taishi/FINCIAL-INSUR.csv')

In [215]:
mama = tocsv(scrplist[0],scrplist[1])

In [216]:
mama.to_csv('C:/Users/GE62/Desktop/realtaishi/FINCIAL-INSUR.csv')

In [ ]:
'Statement of Financial Position (MB.)'

In [217]:

page = requests.get('https://www.set.or.th/set/factsheet.do?symbol=AEC&ssoPageId=3&language=en&country=US')
soup = BeautifulSoup(page.text, 'html.parser')
for z in soup.findAll('table',{'class':'factsheet'}):
    for tr in z.findAll('tr'):
        rstuff=[td for td in tr.stripped_strings]
        print(rstuff)

['Price (B.)', '52 Week High/Low', 'P/E (X)', 'P/BV (X)', 'Paid-up (MB.)', 'Market Cap (MB.)', 'EV (MB.)', 'EBITDA (MB.)', 'EV/EBITDA']
['0.51', '0.65\xa0/\xa00.51', 'N/A', '0.58', '1,224.25', '624.37', '-', '-20.35', '-']
['News']
['Date/Time', 'Subject']
['30 Apr 2018 09:09', "Shareholders meeting's resolution(Revised)"]
['27 Apr 2018 18:02', "Shareholders meeting's resolution"]
['26 Apr 2018 08:17', 'SET reminds the last trading day of AEC-W4 on April 30, 2018']
['20 Apr 2018 07:51', 'Trading suspension and listing status termination of AEC-W4']
['19 Apr 2018 17:01', 'Notification the final exercise and request for suspension of AEC-W4 (Revised)']
['more']
['Company Profile']
['Address', '17th fl., Athenee Tower, 63 Wittayu Road, Lumpini, Pathumwan Bangkok\xa010330']
['Telephone', '0-2659-3456', 'Fax', '0-2659-3457', 'URL', 'www.aecs.com', 'Annual Report 2017', 'Establish Date', '1/4/1971', 'Listed Date', '08/08/2001', 'IPO Price (Baht)', '3.00', 'Baht', 'IPO Financial Advisory', 'I